In [78]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import numpy.typing as npt

In [79]:
df = pd.read_csv("Gemini_BTCUSD_1h.csv", parse_dates=["date"])

In [80]:
def generate_label_window(
        closings: npt.ArrayLike, dim: int, lookahead: int
    ) -> tuple[npt.ArrayLike, float]:
    window = closings[:dim]
    window_last_price = window[-1]
    with_lookahead = closings[:dim + lookahead]

    stddev_half = np.std(with_lookahead) * 0.5

    buy = np.abs(np.min(with_lookahead) - window_last_price) < stddev_half 
    sell = np.abs(np.max(with_lookahead) - window_last_price) < stddev_half

    if buy == sell:
        return 0
    
    return -1 if sell else 1

In [84]:
closings = np.flip(np.array(df["close"][:17520]))
dim = 48
lookahead = 24

sells_x, sells_y = [], []
buys_x, buys_y = [], []

for win_start in range(len(closings) - dim - lookahead):
  xTr = closings[win_start:][:dim + lookahead]
  yTr = generate_label_window(xTr, dim, lookahead)

  win_x = win_start + dim
  win_y = closings[win_x]

  if yTr == 1 and len(buys_x) == len(sells_x):
    buys_x.append(win_x)
    buys_y.append(win_y)
  elif yTr == -1 and len(buys_x) > len(sells_x):
    sells_x.append(win_x)
    sells_y.append(win_y)

if len(buys_x) > len(sells_x):
  buys_x.pop()
  buys_y.pop()

fee_keep = (1 - 0.2/100) ** (2 * len(sells_y))
profit = np.sum(np.array(sells_y) / np.array(buys_y) - 1) * 100 * fee_keep
buy_hold = (closings[-1] / closings[0] - 1) * 100

print("Profit: " + str(profit) + "%")
print("Buy + Hold: " + str(buy_hold) + "%")
print("Start End:", closings[0], closings[-2])
print("% kept after fees: " + str(fee_keep * 100) + "%")

"""
plt.scatter(sells_x, sells_y, c="red")
plt.scatter(buys_x, buys_y, c="green")
plt.plot(closings)
plt.ylabel("USD/BTC")
plt.xlabel("Time since " + str(df["date"][0]))
plt.show()
"""

Profit: 319.5927508602918%
Buy + Hold: -49.80467701119691%
Start End: 54932.09 27584.69
% kept after fees: 49.82282720379108%


'\nplt.scatter(sells_x, sells_y, c="red")\nplt.scatter(buys_x, buys_y, c="green")\nplt.plot(closings)\nplt.ylabel("USD/BTC")\nplt.xlabel("Time since " + str(df["date"][0]))\nplt.show()\n'